# Import Packages

In [1]:
# import packages
from gurobipy import *
import numpy as np
import pandas as pd

# Parameters

In [2]:
# Need input
input_file_name = "flat1000_76_0_upper_Q_m2"
file_type = ".txt"
file_path = ""

In [3]:
output_file_path = ""

In [4]:
timeLimit = 3600 * 24
preQLin_val = 0
logToFile = 1    # 1 for logging to file or else 0

# Read Data

In [5]:
with open (file_path + input_file_name + file_type, "r") as file:
    count = 0
    for line in file:
        line = line.split()
        #print(line)
        if (count == 2):
            n = int(line[2])
            K = int(line[3])
            p = int(line[4])
            Q = np.zeros((n*K,n*K))
        elif (count >= 6):
            i = int(line[0])
            j = int(line[1])
            value = float(line[2])
            
            Q[i-1][j-1] = -value
                    
        count = count + 1

# Build Model

In [ ]:
# Create a new model
m1 = Model("MSCP3")

# Logging
if logToFile == 1:
    m1.Params.LogFile = output_file_path + input_file_name + "_m2_" + str(preQLin_val) + "_24hrs.log"
    m1.Params.LogToConsole = 0

# Create variables
X = m1.addMVar(n*K, vtype=GRB.BINARY, name="X")

# Objective function
obj_func = X @ Q @ X

# Constraint
for i in range(0,n):
    m1.addConstr(X[i*K:(i+1)*K].sum() == 1)

# preprocessing type
m1.Params.PreQLinearize= preQLin_val

# Set objective
m1.setObjective(obj_func, GRB.MINIMIZE)

# Time limit
m1.Params.timelimit = timeLimit

# Optimize model
m1.optimize()

if logToFile == 1:
    m1.Params.LogFile = ""